In [ ]:
using Plots
using FFTW

L = 100
N = 1000
dx = L/N
dt = 1e-2

x = LinRange(0, L - dx, N)
k = 2*π/L .* collect(1:(N÷2)+1)
k² = k.^2
c = -k.^4
k1 = exp.(c .* dt)
k2 = dt * ( exp.(c .* dt) .- 1 ) ./ (dt * c)

F = plan_rfft(x)
B = plan_irfft(k.*im, N)

g(φ) = k² .* (F * (φ .- φ.^3))

function antialiasing!(F)
    F[end-(N÷6)+1:end] .= 0.
end

function etd!(Fφ, φ)
    Fφ .= k1 .* Fφ + k2 .* g(φ)
    antialiasing!(Fφ)
    φ .= B*Fφ
end

In [ ]:
φ = cos.(2  * (2π/L) .* x)  * 0.1

dφ = zeros(N)
Fφ = F*φ 
Fdφ = F*dφ 
c = "lightblue"
p = plot(x, φ, color=c)
display(p)

In [ ]:
M = 10_000
p = plot(x, φ, color=c)
for t in 1:M
    etd!(Fφ, φ)
    if t%(M÷10) == 0
        plot!(x, φ; legend=false, color=c)
        @assert !any(isnan.(Fφ))
    end
end

In [ ]:
display(p)

In [ ]:
dt2 = 1e-5
function euler!(Fdφ, Fφ, φ)
    Fdφ .= -k² .* (k².*Fφ .- F * (φ .- φ.^3 ))
    Fφ .+= Fdφ.*dt2
    antialiasing!(Fφ)
    φ .= B*Fφ
end

In [ ]:
φ2 = cos.(2  * (2π/L) .* x)  * 0.1

dφ2 = zeros(N)
Fφ2 = F*φ2
Fdφ2 = F*dφ2 

M2 = M*1000
p = plot(x, φ2, color=c)
for t in 1:M2
    euler!(Fdφ2, Fφ2, φ2)
    if t%(M2÷10) == 0
        print(100*t/M2,"% \n")
        plot!(x, φ2, color=c)
        @assert !any(isnan.(Fφ))
    end
end
display(p)

In [ ]:
plot(x, φ2, color=c)

In [ ]:
plot(x, φ, color=c)

In [ ]:
plot(x, φ - φ2, color=c)

In [ ]:
@userplot Evolve
@recipe function f(ev::Evolve)
    x, φ, Fφ = ev.args
    for i in 1:50
        etd!(Fφ, φ)
    end
    yrange --> (-1, 1)
    x, φ
end


φ = cos.(2  * (2π/L) .* x)  * 1e-1
Fφ = F*φ 
M = 500

anim = @animate for i ∈ 1:M
    evolve(x, φ, Fφ)
end
gif(anim, "anim_fps15.mp4", fps = 60)


In [ ]:
@userplot Evolve
@recipe function f(ev::Evolve)
    x, φ, Fφ = ev.args
    for i in 1:4000
        etd!(Fφ, φ)
    end
    yrange --> (-1, 1)
    x, φ
end


φ = cos.(2  * (2π/L) .* x)  * 1e-5
Fφ = F*φ
M = 100
anim = @animate for i ∈ 1:M
    evolve(x, φ, Fφ)
end
gif(anim, "anim_fps2.mp4", fps = 30)